# Validación del Umbral de Amortiguamiento Geométrico

Este notebook permite validar los parámetros críticos de la demostración de regularidad global para las ecuaciones de Navier-Stokes 3D.

## Parámetros clave

- **δ\*** (delta star): Defecto de desalineación = (a² c₀²)/(4π²)
- **γ** (gamma): Coeficiente de amortiguamiento = ν c* - (1 - δ*/2) C_str

### Condición crítica

Para que la desigualdad de Riccati cierre y se garantice regularidad global, se requiere:

**γ > 0**

Esto implica que δ* debe ser suficientemente grande.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
rc('font', size=12)
rc('axes', labelsize=12, titlesize=14)
rc('legend', fontsize=10)

## Definición de funciones

In [ ]:
def delta_star(a, c0):
    """
    Calcula el defecto de desalineación.
    
    Parameters:
    -----------
    a : float
        Parámetro de amplitud
    c0 : float
        Gradiente de fase
    
    Returns:
    --------
    float
        Defecto de desalineación δ*
    """
    return (a**2 * c0**2) / (4 * np.pi**2)


def gamma(nu, a, c0, c_star=1/16, C_str=32):
    """
    Calcula el coeficiente de amortiguamiento.
    
    Parameters:
    -----------
    nu : float
        Viscosidad cinemática
    a : float
        Parámetro de amplitud
    c0 : float
        Gradiente de fase
    c_star : float, optional
        Coeficiente de coercividad parabólica (default: 1/16)
    C_str : float, optional
        Constante de estiramiento de vorticidad (default: 32)
    
    Returns:
    --------
    float
        Coeficiente de amortiguamiento γ
    """
    delta = delta_star(a, c0)
    return nu * c_star - (1 - delta/2) * C_str

## Análisis de parámetros actuales

In [ ]:
# Parámetros por defecto
a_default = 7.0
c0_default = 1.0
nu_default = 1e-3

# Calcular δ* y γ con parámetros actuales
delta_default = delta_star(a_default, c0_default)
gamma_default = gamma(nu_default, a_default, c0_default)

print("="*60)
print("ANÁLISIS DE PARÁMETROS ACTUALES")
print("="*60)
print(f"\nParámetros:")
print(f"  a = {a_default}")
print(f"  c₀ = {c0_default}")
print(f"  ν = {nu_default}")
print(f"\nResultados:")
print(f"  δ* = {delta_default:.6f}")
print(f"  γ = {gamma_default:.6f}")
print(f"\nEstado:")
if gamma_default > 0:
    print(f"  ✅ γ > 0: La desigualdad de Riccati CIERRA")
else:
    print(f"  ❌ γ ≤ 0: La desigualdad de Riccati NO CIERRA")
    print(f"  ⚠️  Se requiere aumentar el parámetro 'a'")
print("="*60)

## Gráfico: γ vs a (variando amplitud)

In [ ]:
# Rango de valores de a
a_vals = np.linspace(1, 300, 300)
gamma_vals = [gamma(nu_default, a, c0_default) for a in a_vals]

# Crear gráfico
fig, ax = plt.subplots(figsize=(10, 6))

# Línea principal
ax.plot(a_vals, gamma_vals, 'b-', linewidth=2, label='γ(a)')

# Línea de referencia en γ=0
ax.axhline(0, color='red', linestyle='--', linewidth=1.5, label='γ = 0 (umbral crítico)')

# Marcar el valor actual
ax.plot(a_default, gamma_default, 'ro', markersize=10, 
        label=f'Actual: a={a_default}, γ={gamma_default:.3f}')

# Encontrar y marcar el punto de cruce
# Buscar el primer índice donde γ > 0
positive_indices = np.where(np.array(gamma_vals) > 0)[0]
if len(positive_indices) > 0:
    a_critical = a_vals[positive_indices[0]]
    ax.axvline(a_critical, color='green', linestyle=':', linewidth=1.5, 
               label=f'a crítico ≈ {a_critical:.1f}')
    ax.plot(a_critical, 0, 'go', markersize=10)

# Etiquetas y título
ax.set_xlabel('a (parámetro de amplitud)', fontsize=14)
ax.set_ylabel('γ (coeficiente de amortiguamiento)', fontsize=14)
ax.set_title(f'Umbral para γ > 0\n(ν = {nu_default}, c₀ = {c0_default})', fontsize=16)
ax.grid(True, alpha=0.3)
ax.legend(loc='best')

# Añadir zona de color
ax.fill_between(a_vals, gamma_vals, 0, where=(np.array(gamma_vals) > 0), 
                 alpha=0.2, color='green', label='Región γ > 0 (válida)')
ax.fill_between(a_vals, gamma_vals, 0, where=(np.array(gamma_vals) <= 0), 
                 alpha=0.2, color='red', label='Región γ ≤ 0 (no válida)')

plt.tight_layout()
plt.show()

## Análisis de sensibilidad: γ vs ν (variando viscosidad)

In [ ]:
# Rango de viscosidades
nu_vals = np.logspace(-4, -2, 100)  # 10^-4 a 10^-2

# Calcular γ para diferentes valores de a
a_test_values = [7.0, 50.0, 100.0, 200.0]

fig, ax = plt.subplots(figsize=(10, 6))

for a_test in a_test_values:
    gamma_vals_nu = [gamma(nu, a_test, c0_default) for nu in nu_vals]
    ax.plot(nu_vals, gamma_vals_nu, linewidth=2, label=f'a = {a_test}')

# Línea de referencia
ax.axhline(0, color='red', linestyle='--', linewidth=1.5, alpha=0.7)

ax.set_xscale('log')
ax.set_xlabel('ν (viscosidad cinemática)', fontsize=14)
ax.set_ylabel('γ (coeficiente de amortiguamiento)', fontsize=14)
ax.set_title('Sensibilidad de γ a la viscosidad\npara diferentes valores de a', fontsize=16)
ax.grid(True, alpha=0.3, which='both')
ax.legend(loc='best')

plt.tight_layout()
plt.show()

## Mapa de calor: γ en función de (a, ν)

In [ ]:
# Crear malla de parámetros
a_range = np.linspace(1, 250, 100)
nu_range = np.logspace(-4, -2, 100)
A, Nu = np.meshgrid(a_range, nu_range)

# Calcular γ para cada combinación
Gamma = np.zeros_like(A)
for i in range(len(nu_range)):
    for j in range(len(a_range)):
        Gamma[i, j] = gamma(Nu[i, j], A[i, j], c0_default)

# Crear gráfico
fig, ax = plt.subplots(figsize=(12, 8))

# Contour plot
contour = ax.contourf(A, Nu, Gamma, levels=20, cmap='RdYlGn', extend='both')
contour_lines = ax.contour(A, Nu, Gamma, levels=[0], colors='black', linewidths=2)
ax.clabel(contour_lines, inline=True, fontsize=10, fmt='γ = %.1f')

# Marcar punto actual
ax.plot(a_default, nu_default, 'r*', markersize=20, 
        label=f'Parámetros actuales (a={a_default}, ν={nu_default})')

# Configuración de ejes
ax.set_xlabel('a (amplitud)', fontsize=14)
ax.set_ylabel('ν (viscosidad)', fontsize=14)
ax.set_yscale('log')
ax.set_title('Mapa de γ en el espacio de parámetros (a, ν)', fontsize=16)
ax.legend(loc='upper left')

# Colorbar
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('γ (coeficiente de amortiguamiento)', fontsize=12)

plt.tight_layout()
plt.show()

## Cálculo del valor crítico de a

In [ ]:
def find_critical_a(nu, c0=1.0, c_star=1/16, C_str=32):
    """
    Encuentra el valor crítico de a para el cual γ = 0.
    
    De la ecuación: γ = ν·c* - (1 - δ*/2)·C_str = 0
    Despejando a:
    
    δ* = 2·(1 - ν·c*/C_str)
    a = sqrt(4·π²·δ*/c0²)
    """
    delta_critical = 2 * (1 - nu * c_star / C_str)
    
    if delta_critical < 0:
        return None  # No hay solución positiva
    
    a_critical = np.sqrt(4 * np.pi**2 * delta_critical / c0**2)
    return a_critical


# Calcular para diferentes viscosidades
print("\n" + "="*60)
print("VALORES CRÍTICOS DE 'a' PARA γ = 0")
print("="*60)

test_viscosities = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]

for nu_test in test_viscosities:
    a_crit = find_critical_a(nu_test, c0_default)
    if a_crit is not None:
        gamma_check = gamma(nu_test, a_crit, c0_default)
        delta_check = delta_star(a_crit, c0_default)
        print(f"\nν = {nu_test:.1e}:")
        print(f"  a_crítico = {a_crit:.2f}")
        print(f"  δ* = {delta_check:.6f}")
        print(f"  γ = {gamma_check:.6e} (debe ser ≈ 0)")
    else:
        print(f"\nν = {nu_test:.1e}: No existe solución (ν muy grande)")

print("\n" + "="*60)

## Herramienta interactiva

Puedes modificar los siguientes valores para explorar el espacio de parámetros:

In [ ]:
# MODIFICA ESTOS VALORES PARA EXPLORAR
a_user = 200.0    # Prueba con diferentes valores de a
c0_user = 1.0     # Gradiente de fase
nu_user = 1e-3    # Viscosidad

# Calcular resultados
delta_user = delta_star(a_user, c0_user)
gamma_user = gamma(nu_user, a_user, c0_user)

print("\n" + "="*60)
print("RESULTADOS CON PARÁMETROS PERSONALIZADOS")
print("="*60)
print(f"\nParámetros de entrada:")
print(f"  a = {a_user}")
print(f"  c₀ = {c0_user}")
print(f"  ν = {nu_user}")
print(f"\nResultados calculados:")
print(f"  δ* = {delta_user:.6f}")
print(f"  γ = {gamma_user:.6f}")
print(f"\nVerificación:")
if gamma_user > 0:
    print(f"  ✅ γ > 0: CONDICIÓN SATISFECHA")
    print(f"  ✅ La desigualdad de Riccati CIERRA")
    print(f"  ✅ Se garantiza regularidad global")
else:
    print(f"  ❌ γ ≤ 0: CONDICIÓN NO SATISFECHA")
    print(f"  ❌ La desigualdad de Riccati NO CIERRA")
    print(f"  ⚠️  La prueba es CONDICIONAL con estos parámetros")
    a_needed = find_critical_a(nu_user, c0_user)
    if a_needed is not None:
        print(f"  💡 Valor mínimo requerido: a ≥ {a_needed:.2f}")
print("="*60)

## Conclusiones

Este notebook demuestra que:

1. **Con los parámetros actuales** (`a = 7.0`, `c₀ = 1.0`, `ν = 10⁻³`):
   - δ* ≈ 0.025
   - γ < 0
   - La desigualdad de Riccati **NO CIERRA**

2. **Para cerrar la prueba**, se requiere:
   - Aumentar `a` hasta aproximadamente **200** (para `ν = 10⁻³`)
   - O ajustar dinámicamente `a` en función de `ν`

3. **La prueba es actualmente CONDICIONAL**:
   - Depende de la elección correcta de parámetros
   - No es universal con los valores por defecto

### Próximos pasos

- Optimización paramétrica para encontrar valores óptimos de `a` y `c₀`
- Validación numérica con simulaciones DNS
- Exploración de reparametrizaciones que absorban `a` como constante universal
- Implementación de módulo espectral dinámico: `a_eff = a(ν)`